In [13]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import dill
import re
import time
from selenium import webdriver

from winespectator_login import winespectator_login_name, winespectator_password

Get the list of daily pick URLs

In [25]:
base_url = 'http://www.winespectator.com/dailypicks/category/catid/1/page/{}'

url_list = list()

for pg in range(1,867):
    url = base_url.format(pg)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    
    soup_list = soup.find_all(attrs={'class':'daily-wine-items'})
    
    if len(soup_list) > 0:
        for sl in soup_list:
            try:
                url_list.append(sl.find('a',href=True).get('href'))
            except:
                pass

In [4]:
len(url_list)

4280

In [29]:
with open('../pkl/01_winespectator_dot_com_url_list.pkl','w') as fh:
    dill.dump(url_list, fh)

Extract individual articles from the urls.

In [21]:
# ssh -ND 8081 nestanmr
# ssh -ND 8082 backupsy
# ssh -ND 8083 aws
# ssh -ND 8084 vultr
phantom_path = '/Volumes/Files/homebrew/bin/phantomjs'

service_args1 = ['--proxy=127.0.0.1:8081', '--proxy-type=socks5']
service_args2 = ['--proxy=127.0.0.1:8082', '--proxy-type=socks5']
service_args3 = ['--proxy=127.0.0.1:8083', '--proxy-type=socks5']
service_args4 = ['--proxy=127.0.0.1:8084', '--proxy-type=socks5']

driver1 = webdriver.PhantomJS(phantom_path, service_args=service_args1)
driver2 = webdriver.PhantomJS(phantom_path, service_args=service_args2)
driver3 = webdriver.PhantomJS(phantom_path, service_args=service_args3)
driver4 = webdriver.PhantomJS(phantom_path, service_args=service_args4)

driver_list = [driver1, driver2, driver3, driver4]

for driver in driver_list:

    driver.get('https://www.winespectator.com/auth/login')

    userid = driver.find_element_by_name('userid')
    userid.send_keys(winespectator_login_name)

    passwd = driver.find_element_by_name('passwd')
    passwd.send_keys(winespectator_password)

    login = driver.find_element_by_id('target')
    login.click()

    time.sleep(1.5)

In [ ]:
wine_df_list = list()

for url in url_list[:10]:
    driver = np.random.choice(driver_list)
    full_url = 'http://www.winespectator.com/' + url
    driver.get(full_url)
    time.sleep(2)
    
    html = driver.page_source
    
    try:
        soup = BeautifulSoup(html, 'lxml')
        wine_data = soup.find(attrs={'class':'mod-container'})
    except:
        pass
    
    # extract winery name
    try:
        winery = wine_data.find('h1').text.strip()
    except:
        winery = ''
    
    # extract wine name
    try:
        wine = wine_data.find('h4').text.strip()
    except:
        pass

    # extract year
    try:
        year = re.search(r"""((?:20|19)[0-9]{2})""", wine).group(1)
        wine = wine.replace(year, '').strip()
    except:
        year = ''
        
    # extract review
    try:
        review = wine_data.find(attrs={'id':'bt-body'}).text.strip()
    except:
        review = ''
        
    # score
    try:
        score = wine_data.find(text=re.compile('Score: [0-9]{2}'))
        score = score.replace('Score:', '').strip()
    except:
        score = ''
        
        
    for para in wine_data.find_all(attrs={'class':'paragraph'}):
        text = para.text

        if 'Release Price' in text:
            try:
                release_price = re.search(r"""Release Price \$([0-9\.]+)""", text).group(1)
            except:
                release_price = ''
        elif 'Country' in text:
            try:
                country = re.search(r"""Country (.+)""", text).group(1)
            except:
                country = ''
        elif 'Region' in text:
            try:
                region = re.search(r"""Region (.+)""", text).group(1)
            except:
                region = ''
    

    df = pd.DataFrame({'winery':winery, 'wine':wine, 'year':year, 'score':score,
                  'price':release_price, 'country':country, 'region':region,
                  'review':review}, index=pd.Index([0]))
    wine_df_list.append(df)

In [ ]:
wine_df = pd.concat(wine_df_list, axis=0).reset_index(drop=True)

In [ ]:
wine_df.to_pickle('../pkl/01_winespectator_dot_com_data.pkl')